In [3]:
# !表示在 jupyter 中运行终端命令
!nvidia-smi  

Tue Jun 10 10:43:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.52                 Driver Version: 576.52         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   43C    P8             31W /  390W |     460MiB /  24576MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# 所有深度学习框架默认在 CPU 上运算, 故需指定 GPU
# 计算设备
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda')

(device(type='cpu'), <torch.cuda.device at 0x1f54c3d5358>)

In [5]:
# 查询可用 GPU 的数量
torch.cuda.device_count()

1

In [10]:
# 这两个函数允许我们在请求的 GPU 不存在的情况下运行代码
def try_gpu(i=0):
    """ 如果存在，则返回gpu()，否则返回cpu() """
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """ 返回所有可用的GPU，若无，返回CPU """
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(1), try_all_gpus()    

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [11]:
# 查询张量所在的设备, 默认在 CPU 
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [18]:
# 存储在 GPU 上
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [19]:
# 第二个 GPU 上创建一个随机张量, 因为我们只有一个 GPU, 所以会创建在 CPU 上
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.0389, 0.3327, 0.8760],
        [0.1615, 0.1906, 0.6167]])

In [22]:
# 要计算 X + Y, 我们需要决定在哪里执行这个操作
Z = Y.cuda(0)
print(Y)
print(Z)

tensor([[0.0389, 0.3327, 0.8760],
        [0.1615, 0.1906, 0.6167]])
tensor([[0.0389, 0.3327, 0.8760],
        [0.1615, 0.1906, 0.6167]], device='cuda:0')


In [23]:
# 现在数据在同一个 GPU 上（X 和 Z）, 我们可以将他们相加
X + Z

tensor([[1.0389, 1.3327, 1.8760],
        [1.1615, 1.1906, 1.6167]], device='cuda:0')

In [25]:
Z.cuda() is Z  # 若 Z 已经在 GPU, 出于性能考虑, 不会在 copy

True

In [26]:
# 神经网络与GPU
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)  # X 与 网络的参数均在GPU

tensor([[0.6601],
        [0.6601]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [27]:
# 确认模型参数存储在同一个GPU上
net[0].weight.data.device

device(type='cuda', index=0)